# Jupyter Notebook accompanying "Future global green value chains: estimating the renewables pull and understanding its impact on industrial relocation"

This Jupyter notebook accompanies the working paper "Future global green value chains: estimating the renewables pull and understanding its impact on industrial relocation" by P.C. Verpoort et al (2023). It contains all the necessary code for reproducing the basic results reported in the manuscript.

For reproducing the results with adjusted assumptions on electricity prices and transport costs, you may also want to refer to the interactive webapp: https://interactive.pik-potsdam.de/green-value-chains/
For full details on how figures were derived, please refer to the full source code on GitHub: https://github.com/PhilippVerpoort/green-value-chains/

### Dependencies
In order to reproduce the data, you will need the following dependencies and data.

For the techno-economic data, you need the Python package `posted` (the Potsdam Open-Source Techno-Economic Database). You can install it as follows:
```bash
# when using poetry
poetry add git+https://git@github.com:PhilippVerpoort/posted.git#v0.2.0

# when using pip
pip install git+https://git@github.com:PhilippVerpoort/posted.git#v0.2.0
```

For all other assumptions, you need the subdirectory `data` and its contents from [here](https://github.com/PhilippVerpoort/green-value-chains/tree/release/data) in your working directory. The easiest way might be to just clone the whole directory with git:

```bash
git clone https://github.com/PhilippVerpoort/green-value-chains.git
```

For plotting, this notebook uses `plotly`:
```bash
# for plotting only
poetry add plotly
```

### Preparation

##### Ignore warnings

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

##### Import IPython tools

In [2]:
from IPython.display import display, Markdown, Latex

##### Set unit registry for pint

In [3]:
import pint
from posted.units.units import ureg
pint.set_application_registry(ureg)

### Loading assumptions

In [4]:
import pathlib
import yaml
import pandas as pd


BASE_PATH = pathlib.Path.cwd()

def loadCSVDataFile(fname: str):
    path = BASE_PATH / 'data' / f"{fname}.csv"
    return pd.read_csv(path)

def loadYAMLDataFile(fname: str):
    path = BASE_PATH / 'data' / f"{fname}.yml"
    with open(path, 'r') as f:
        ret = yaml.load(f.read(), Loader=yaml.FullLoader)
    return ret

In [5]:
inputs = {}

# load value chain definitions
inputs['value_chains'] = loadYAMLDataFile('value_chains')

# load data for electricity-price cases
inputs['epdcases'] = loadCSVDataFile('epd_cases') \
    .astype({c: 'pint[EUR/MWh]' for c in ('RE-scarce', 'RE-rich')}, errors='ignore')

# load data for other prices
inputs['other_prices'] = loadCSVDataFile('other_prices') \
    .astype({'assump': 'float32'}, errors='ignore') \
    .set_index(['type', 'unit']) \
    .transpose() \
    .pint.quantify() \
    .reset_index(drop=True)

# load data for specific transport cost
inputs['transp_cost'] = loadCSVDataFile('transp_cost') \
    .astype({'assump': 'float32'}, errors='ignore')

# load other assumptions
inputs['other_assump'] = loadYAMLDataFile('other_assump')

# load scenarios and volumes
inputs['scenarios'] = loadCSVDataFile('scenarios').set_index(['scenario', 'commodity'])
inputs['volumes'] = loadCSVDataFile('volumes').astype({'volume': 'float32'}).set_index(['commodity'])['volume']

##### Electricity-price cases

In [6]:
inputs['epdcases'].set_index(['epdcase', 'process'])

RE-rich  RE-scarce
epdcase process                    
weak    ELH2          30         50
        OTHER         50         70
medium  ELH2          30         70
        OTHER         50         90
strong  ELH2          15         85
        OTHER         35        105

##### Other price data

In [7]:
inputs['other_prices'].iloc[0].to_frame('value')

,value
type,
price:ng,45.0 EUR/MWh
price:ironore,130.0 EUR/t
price:alloys,1720.0 EUR/t
price:coal,4.0 EUR/GJ
price:graph_electr,4000.0 EUR/t
price:lime,100.0 EUR/t
price:nitrogen,100.0 EUR/t
price:oxygen,100.0 EUR/t
price:steelscrap,180.0 EUR/t


##### Transport cost data

In [8]:
inputs['transp_cost'].set_index(['traded', 'impsubcase'])

assump     unit
traded   impsubcase                 
h2       A             50.0  EUR/MWh
         B             15.0  EUR/MWh
ironore  NaN           10.0    EUR/t
dri      NaN           20.0    EUR/t
steelhrc NaN           20.0    EUR/t
nh3      NaN           30.0    EUR/t
urea     NaN           50.0    EUR/t
meoh     NaN           30.0    EUR/t
hvcs     NaN           80.0    EUR/t

##### Other assumptions

In [9]:
print(yaml.dump(inputs['other_assump']))

irate:
  RE-rich: 8.0
  RE-scarce: 5.0
ltime: 18
ocf:
  ELH2: 0.5
  default: 0.95
period: 2040



##### Structure of the value chains

In [10]:
print(yaml.dump(inputs['value_chains'], sort_keys=False))

Steel:
  graph:
    HOTROLL:
      steelslab: CAST
    CAST:
      steelliq: EAF
    EAF:
      dri: IDR
    IDR:
      h2: ELH2
    ELH2: {}
  locations:
  - - ELH2
  - - IDR
  - - EAF
    - CAST
    - HOTROLL
Urea:
  graph:
    UREA-SYN:
      nh3: HBNH3-ASU
      co2: DAC
    HBNH3-ASU:
      h2: ELH2
    DAC:
      heat: HEATPUMP-4-DAC
    HEATPUMP-4-DAC: {}
    ELH2: {}
  locations:
  - - ELH2
  - - HBNH3-ASU
  - - HEATPUMP-4-DAC
    - DAC
    - UREA-SYN
Ethylene:
  graph:
    MEOH-2-OLEF:
      meoh: MEOH-SYN
    MEOH-SYN:
      h2: ELH2
      co2: DAC
    DAC:
      heat: HEATPUMP-4-DAC
    HEATPUMP-4-DAC: {}
    ELH2: {}
  locations:
  - - ELH2
  - - HEATPUMP-4-DAC
    - DAC
    - MEOH-SYN
  - - MEOH-2-OLEF



### Loading techno-economic data from POSTED

In [11]:
from posted.ted.Mask import Mask
from posted.ted.TEDataSet import TEDataSet


# create list of technologies to load
vcs = inputs['value_chains']
techs = {k: {} for comm in vcs for k in vcs[comm]['graph'].keys()}

# add settings for loading the data
techs['ELH2'] |= {
    'subtech': 'Alkaline',
    'masks': [
        Mask(
            when="type.str.startswith('capex')",
            use=f"src_ref.isin({'Vartiainen et al. (2022)', 'IRENA Global Hydrogen trade costs (2022)'})",
        ),
        Mask(
            when="type.str.startswith('demand:elec')",
            use=f"src_ref.isin({'Vartiainen et al. (2022)', 'IRENA Global Hydrogen trade costs (2022)'})",
        ),
    ],
}
techs['DAC'] |= {
    'subtech': 'LT-DAC',
    'masks': [
        Mask(
            when="type.str.startswith('capex')",
            use="src_ref=='custom'",
        )
    ]
}
techs['IDR'] |= {'mode': 'h2'}
techs['EAF'] |= {'mode': 'primary'}

# load datatables from POSTED
inputs['proc_tables'] = {}
for tid, kwargs in techs.items():
    dac = {'load_other': ['data/DAC-capex-custom.csv'], 'load_database': True} if tid == 'DAC' else {}
    t = TEDataSet(tid, **dac).generateTable(
        period=inputs['other_assump']['period'],
        agg=['src_ref'],
        **kwargs,
    )
    inputs['proc_tables'][tid] = t

/home/philippv/Documents/4-projects/02-green-value-chain/01-vcs/green-value-chains/.venv/lib/python3.11/site-packages/posted/ted/TEDataSet.py:446: TEGenerationFailure: No CAPEX value matching a relative FOPEX value found.
         type flow_type           unit component value
21  fopex_rel       NaN  dimensionless       NaN  0.04 (TID: DAC)
  warnings.warn(TEGenerationFailure(rows.loc[cond], self._tid, 'No CAPEX value matching a relative FOPEX value found.'))
/home/philippv/Documents/4-projects/02-green-value-chain/01-vcs/green-value-chains/.venv/lib/python3.11/site-packages/posted/ted/TEDataSet.py:446: TEGenerationFailure: No CAPEX value matching a relative FOPEX value found.
         type flow_type           unit component value
22  fopex_rel       NaN  dimensionless       NaN  0.04 (TID: DAC)
  warnings.warn(TEGenerationFailure(rows.loc[cond], self._tid, 'No CAPEX value matching a relative FOPEX value found.'))


In [12]:
pd.concat([
        table.data.assign(tech=tid).reset_index(drop=(tid!='EAF'))
        for tid, table in inputs['proc_tables'].items()
    ]) \
    .fillna({'reheating': ''}) \
    .set_index(['tech', 'reheating'])

part                          value                                           \
type                          capex demand:elec demand:heat demand:steelslab   
tech           reheating                                                       
HOTROLL                       103.3     0.09432      0.4147            1.053   
CAST                          41.63    0.009317    0.007222              NaN   
EAF            w/ reheating   234.8      0.6129      0.1667              NaN   
               w/o reheating  234.8      0.5731      0.1667              NaN   
IDR                           320.7     0.09049      0.4839              NaN   
ELH2                          31.01       1.411         NaN              NaN   
UREA-SYN                      213.0      0.1326      0.9139              NaN   
HBNH3-ASU                     446.2       0.824         NaN              NaN   
DAC                           173.5       1.052       2.261              NaN   
HEATPUMP-4-DAC                66.61      0.3045         NaN              NaN   
MEOH-2-OLEF                   395.1       1.389         NaN              NaN   
MEOH-SYN                      355.3        2.14         NaN              NaN   

part                                                                    \
type                         demand:steelliq demand:alloys demand:coal   
tech           reheating                                                 
HOTROLL                                  NaN           NaN         NaN   
CAST                                   1.032           NaN         NaN   
EAF            w/ reheating              NaN         0.011      0.1836   
               w/o reheating             NaN         0.011      0.1836   
IDR                                      NaN           NaN         NaN   
ELH2                                     NaN           NaN         NaN   
UREA-SYN                                 NaN           NaN         NaN   
HBNH3-ASU                                NaN           NaN         NaN   
DAC                                      NaN           NaN         NaN   
HEATPUMP-4-DAC                           NaN           NaN         NaN   
MEOH-2-OLEF                              NaN           NaN         NaN   
MEOH-SYN                                 NaN           NaN         NaN   

part                                                                     ...  \
type                         demand:dri demand:graph_electr demand:lime  ...   
tech           reheating                                                 ...   
HOTROLL                             NaN                 NaN         NaN  ...   
CAST                                NaN                 NaN         NaN  ...   
EAF            w/ reheating       1.001            0.002125        0.05  ...   
               w/o reheating      1.001            0.002125        0.05  ...   
IDR                                 NaN                 NaN         NaN  ...   
ELH2                                NaN                 NaN         NaN  ...   
UREA-SYN                            NaN                 NaN         NaN  ...   
HBNH3-ASU                           NaN                 NaN         NaN  ...   
DAC                                 NaN                 NaN         NaN  ...   
HEATPUMP-4-DAC                      NaN                 NaN         NaN  ...   
MEOH-2-OLEF                         NaN                 NaN         NaN  ...   
MEOH-SYN                            NaN                 NaN         NaN  ...   

part                                                                      \
type                         fopex_spec   vopex demand:h2 demand:ironore   
tech           reheating                                                   
HOTROLL                             NaN     NaN       NaN            NaN   
CAST                                NaN     NaN       NaN            NaN   
EAF            w/ reheating       23.89     9.0       NaN            NaN   
               w/o reheating   

### Data processing and calculations

##### Combine individual processes into value chains

In [13]:
from posted.ted.TEProcessTreeDataTable import TEProcessTreeDataTable

inputs['vc_tables'] = {}
for comm, details in vcs.items():
    graph = vcs[comm]['graph']
    t = TEProcessTreeDataTable(*(inputs['proc_tables'][tid] for tid in graph), processGraph=graph)

    # map heat to electricity
    allCols = []
    for idx, cols in t.data.groupby(by=[c for c in t.data.columns.names if c != 'type'], axis=1):
        cols = cols.copy()
        if 'demand:heat' in cols.columns.get_level_values(level='type'):
            cols[(*idx, 'demand:elec')] += cols[(*idx, 'demand:heat')].fillna(0)
            cols = cols.drop(columns=[(*idx, 'demand:heat')])
        allCols.append(cols)
    t.data = pd.concat(allCols, axis=1)

    inputs['vc_tables'][comm] = t

/home/philippv/Documents/4-projects/02-green-value-chain/01-vcs/green-value-chains/.venv/lib/python3.11/site-packages/posted/utils/utils.py:13: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  ret = pd.merge(
/home/philippv/Documents/4-projects/02-green-value-chain/01-vcs/green-value-chains/.venv/lib/python3.11/site-packages/posted/ted/TEProcessTreeDataTable.py:52: PerformanceWarning: indexing past lexsort depth may impact performance.
  for colID in data['value', pDem].columns:
/home/philippv/Documents/4-projects/02-green-value-chain/01-vcs/green-value-chains/.venv/lib/python3.11/site-packages/posted/ted/TEProcessTreeDataTable.py:52: PerformanceWarning: indexing past lexsort depth may impact performance.
  for colID in data['value', pDem].columns:
/home/philippv/Documents/4-projects/02-green-value-chain/01-vcs/green-value-chains/.venv/lib/python3.11/site-packages/posted/ted/TEProcessTreeDataTable.py:52: PerformanceWarning:

In [14]:
for comm, table in inputs['vc_tables'].items():
    display(Markdown(f"**{comm}**"))
    display(table.data.stack('process'))

**Steel**

part                                    value                          \
type                                    capex           demand:alloys   
reheating     process                                                   
w/ reheating  CAST               43.83639 EUR                     NaN   
              EAF      255.15622079999997 EUR  0.011953655999999998 t   
              ELH2      63.61881868598253 EUR                     NaN   
              HOTROLL               103.3 EUR                     NaN   
              IDR       348.8519106071999 EUR                     NaN   
w/o reheating CAST               43.83639 EUR                     NaN   
              EAF      255.15622079999997 EUR  0.011953655999999998 t   
              ELH2      63.61881868598253 EUR                     NaN   
              HOTROLL               103.3 EUR                     NaN   
              IDR       348.8519106071999 EUR                     NaN   

part                                                                    \
type                               demand:coal             demand:elec   
reheating     process                                                    
w/ reheating  CAST                         NaN         0.017415567 MWh   
              EAF      0.19951738559999999 MWh  0.8471882015999999 MWh   
              ELH2                         NaN  2.8947485703296145 MWh   
              HOTROLL                      NaN             0.50902 MWh   
              IDR                          NaN  0.6248115027554397 MWh   
w/o reheating CAST                         NaN         0.017415567 MWh   
              EAF      0.19951738559999999 MWh  0.8039377007999999 MWh   
              ELH2                         NaN  2.8947485703296145 MWh   
              HOTROLL                      NaN             0.50902 MWh   
              IDR                          NaN  0.6248115027554397 MWh   

part                                                             \
type                  demand:graph_electr        demand:ironore   
reheating     process                                             
w/ reheating  CAST                    NaN                   NaN   
              EAF           0.002309229 t                   NaN   
              ELH2                    NaN                   NaN   
              HOTROLL                 NaN                   NaN   
              IDR                     NaN  1.5707582130239994 t   
w/o reheating CAST                    NaN                   NaN   
              EAF           0.002309229 t                   NaN   
              ELH2                    NaN                   NaN   
              HOTROLL                 NaN                   NaN   
              IDR                     NaN  1.5707582130239994 t   

part                                                                    \
type                              demand:lime                demand:ng   
reheating     process                                                    
w/ reheating  CAST                        NaN                      NaN   
              EAF      0.054334799999999996 t  0.23548702319999998 MWh   
              ELH2                        NaN                      NaN   
              HOTROLL                     NaN                      NaN   
              IDR                         NaN   0.5220269158103998 MWh   
w/o reheating CAST                        NaN                      NaN   
              EAF      0.054334799999999996 t  0.23548702319999998 MWh   
              ELH2                        NaN                      NaN   
              HOTROLL                     NaN                      NaN   
              IDR                         NaN   0.5220269158103998 MWh   

part                                                                \
type                        demand:nitrogen          demand:oxygen   
reheating     process                                                
w/ reheating  CAST                      Na

**Urea**

part                          value                           \
type                          capex              demand:elec   
  process                                                      
0 DAC                  129.1187 EUR            0.7828984 MWh   
  ELH2             105.78983975 EUR          4.813591225 MWh   
  HBNH3-ASU             256.565 EUR  0.47379999999999994 MWh   
  HEATPUMP-4-DAC  112.080397282 EUR   0.5123627229000001 MWh   
  UREA-SYN                213.0 EUR               1.0465 MWh   

part                                                                \
type                      demand:water demand_sc:co2  demand_sc:h2   
  process                                                            
0 DAC                              NaN           NaN           NaN   
  ELH2            0.9722703749999999 t           NaN           NaN   
  HBNH3-ASU                        NaN           NaN  3.411475 MWh   
  HEATPUMP-4-DAC                   NaN           NaN           NaN   
  UREA-SYN                         NaN      0.7442 t           NaN   

part                                                                           \
type                      demand_sc:heat demand_sc:nh3             fopex_spec   
  process                                                                       
0 DAC             1.6826362000000001 MWh           NaN          17.756612 EUR   
  ELH2                               NaN           NaN        5.918909125 EUR   
  HBNH3-ASU                          NaN           NaN  6.152499999999999 EUR   
  HEATPUMP-4-DAC                     NaN           NaN        2.035989802 EUR   
  UREA-SYN                           NaN       0.575 t                    NaN   

part                                                       
type             lifetime     ocf                   vopex  
  process                                                  
0 DAC              37.5 a  0.9132                     NaN  
  ELH2                NaN     NaN                     NaN  
  HBNH3-ASU           NaN     NaN  1782.4999999999998 EUR  
  HEATPUMP-4-DAC      NaN     NaN                     NaN  
  UREA-SYN            NaN     NaN                     NaN

**Ethylene**

part                           value                                          \
type                           capex        demand:elec         demand:water   
  process                                                                      
0 DAC                  556.97664 EUR     3.37717248 MWh                  NaN   
  ELH2               455.3967348 EUR    20.72121228 MWh  4.185361799999999 t   
  HEATPUMP-4-DAC  483.4788693504 EUR  2.21016837888 MWh                  NaN   
  MEOH-2-OLEF              395.1 EUR          1.389 MWh                  NaN   
  MEOH-SYN               810.084 EUR         4.8792 MWh                  NaN   

part                                                            \
type                     demand_sc:co2            demand_sc:h2   
  process                                                        
0 DAC                              NaN                     NaN   
  ELH2                             NaN                     NaN   
  HEATPUMP-4-DAC                   NaN                     NaN   
  MEOH-2-OLEF                      NaN                     NaN   
  MEOH-SYN        3.2102399999999998 t  14.685479999999998 MWh   

part                                                                     \
type              demand_sc:heat demand_sc:meoh              fopex_spec   
  process                                                                 
0 DAC             7.25835264 MWh            NaN          76.5963264 EUR   
  ELH2                       NaN            NaN  25.479307799999997 EUR   
  HEATPUMP-4-DAC             NaN            NaN        8.7826066944 EUR   
  MEOH-2-OLEF                NaN         2.28 t               29.85 EUR   
  MEOH-SYN                   NaN            NaN            18.67776 EUR   

part                                                      
type             lifetime     ocf                  vopex  
  process                                                 
0 DAC              37.5 a  0.9132                    NaN  
  ELH2                NaN     NaN                    NaN  
  HEATPUMP-4-DAC      NaN     NaN                    NaN  
  MEOH-2-OLEF         NaN     NaN               20.0 EUR  
  MEOH-SYN         25.0 a     NaN  56.99999999999999 EUR

##### Combine with other assumptions

In [15]:
# add OCF and other prices (all except electricity) to tables
for comm, table in inputs['vc_tables'].items():
    # other prices
    assumpOther = inputs['other_prices']

    # ocf assumptions
    assumpOCF = pd.DataFrame(
        columns=pd.MultiIndex.from_product([table.data.columns.unique('process'), ['ocf']], names=['process', 'type']),
        index=[0],
        data=inputs['other_assump']['ocf']['default'],
    )
    assumpOCF['ELH2', 'ocf'] = inputs['other_assump']['ocf']['ELH2']
    display(assumpOCF)

    # add assumptions to tables
    inputs['vc_tables'][comm] = table \
        .assume(assumpOCF) \
        .assume(assumpOther)

process,CAST,EAF,ELH2,HOTROLL,IDR
type,ocf,ocf,ocf,ocf,ocf
0,0.95,0.95,0.5,0.95,0.95


process,DAC,ELH2,HBNH3-ASU,HEATPUMP-4-DAC,UREA-SYN
type,ocf,ocf,ocf,ocf,ocf
0,0.95,0.5,0.95,0.95,0.95


process,DAC,ELH2,HEATPUMP-4-DAC,MEOH-2-OLEF,MEOH-SYN
type,ocf,ocf,ocf,ocf,ocf
0,0.95,0.5,0.95,0.95,0.95


### Process data

In [16]:
import numpy as np

from posted.calc_routines.LCOX import LCOX


outputs = {}
vcs = inputs['value_chains']

# calculate epd from price cases
outputs['epd'] = inputs['epdcases'] \
    .assign(epd=lambda row: row['RE-scarce'] - row['RE-rich']) \
    .rename_axis('type', axis=1) \
    .sort_values(by='epd')

# calculate transport cost outputs from inputs
outputs['transp_cost'] = inputs['transp_cost'] \
    .set_index(['traded', 'impsubcase', 'unit']) \
    .transpose() \
    .stack('impsubcase') \
    .pint.quantify() \
    .droplevel(0)

# associate correct RE prices with processes
outputs['cases'] = {}
outputs['tables'] = {}
outputs['procLocs'] = {}
outputs['lcox'] = {}
for comm in vcs:
    locs = vcs[comm]['locations']

    # get dataframe of process locations
    procLocs = pd.DataFrame(data=[
            {
                'impcase': f"Case {i}" if i else 'Base Case',
                **{
                    p: loc
                    for loc, pgList in {'RE-rich': locs[:i], 'RE-scarce': locs[i:]}.items()
                    for pg in pgList for p in pg
                }
            }
            for i in range(4)
        ]) \
        .set_index('impcase') \
        .rename_axis('process', axis=1)
    outputs['procLocs'][comm] = procLocs

    # get dataframe mapping epdcases and associated elec prices to processes
    procLocsStacked = procLocs \
        .stack() \
        .swaplevel(0, 1, 0) \
        .to_frame('location') \
        .reset_index() \
        .set_index(['process', 'location'])

    # electricity-price difference cases by process
    epdcasesByProc = pd.concat(
        [inputs['epdcases'].query("process!='OTHER'")]
      + [inputs['epdcases'].query("process=='OTHER'").assign(process=p)
         for p in procLocs.columns if p != 'ELH2']
    )

    outputs['cases'][comm] = epdcasesByProc \
        .set_index(['process', 'epdcase']) \
        .rename_axis('location', axis=1) \
        .stack() \
        .to_frame('price:elec') \
        .merge(procLocsStacked, left_index=True, right_index=True) \
        .reset_index() \
        .drop(columns='location') \
        .set_index(['impcase', 'epdcase', 'process']) \
        .rename_axis('type', axis=1) \
        .unstack('process')

    # financing assumptions
    assumpWACC = procLocs \
        .replace('RE-scarce', inputs['other_assump']['irate']['RE-scarce']) \
        .replace('RE-rich', inputs['other_assump']['irate']['RE-rich']) \
        .astype(float) \
        .apply(lambda x: x/100.0) \
        .assign(type='wacc') \
        .set_index('type', append=True) \
        .unstack('type')
    table = inputs['vc_tables'][comm] \
        .assume(assumpWACC) \
        .assume({'lifetime': 18.0 * ureg('a')})

    # insert dummy process
    newData = table.data.copy()
    newData['value', f"demand_sc:{table.refFlow}", 'DUMMY'] = 1.0
    table.data = newData

    # find goods in value chain that have transport cost
    traded = [
        t.split(':')[-1] for t in outputs['transp_cost'].columns
        if table.data.columns.unique(level=1).str.match(fr"^demand(_sc)?:{t.split(':')[-1]}$").any()
    ]

    # create dataframe containing transport cost assumptions
    assumpTransp = pd.DataFrame(
        index=[f"Case {i}" if i else 'Base Case' for i in range(4)],
        columns=traded,
        data=np.nan,
    ) \
    .rename_axis('impcase') \
    .rename_axis('traded', axis=1)

    # match traded goods to import cases
    for p1, p1s in vcs[comm]['graph'].items():
        for t, p2 in p1s.items():
            if t in traded:
                assumpTransp.loc[(procLocs[p1] != procLocs[p2]), t] = 1
    assumpTransp.loc['Case 3', table.refFlow] = 1
    if comm == 'Steel':
        assumpTransp.loc[['Base Case', 'Case 1'], 'ironore'] = 1

    # determine trade cost cases (including impsubcases)
    tradeCostCases = []
    for impcase, row in assumpTransp.iterrows():
        tradeCostCase = pd.DataFrame(columns=['impcase'], data=[impcase])
        for t in row.dropna().index.tolist():
            tmp = outputs['transp_cost'] \
                .loc[:, t] \
                .dropna() \
                .to_frame() \
                .rename_axis(f"impsubcase_{t}") \
                .reset_index()
            tradeCostCase = tradeCostCase.merge(tmp, how='cross').dropna(axis=1, how='all')
        tradeCostCases.append(tradeCostCase)
    assumpTransp = pd.concat(tradeCostCases)
    indexCols = [c for c in assumpTransp if c not in traded]
    assumpTransp = assumpTransp \
        .fillna({c: '' for c in indexCols}) \
        .set_index(indexCols) \
        .rename(columns={c: f"transp:{c}" for c in traded}) \
        .rename_axis('type', axis=1)
    assumpTransp.index = assumpTransp.index \
        .map(lambda i: (i[0], f"{i[0]}{i[1]}")) \
        .rename(['impcase', 'impsubcase'])

    # add trade cost assumptions to table
    table = table.assume(assumpTransp)

    # associate reheating cases to impcases
    if comm == 'Steel':
        table.data = table.data \
            .query(f"(reheating=='w/o reheating' & impcase!='Case 2') | "
                   f"(reheating=='w/ reheating') & (impcase=='Case 2')") \
            .droplevel(level='reheating')

    outputs['tables'][comm] = table

    # assume electriticy prices and calculate levelised cost
    outputs['lcox'][comm] = table \
        .assume(outputs['cases'][comm]) \
        .calc(LCOX)

/home/philippv/Documents/4-projects/02-green-value-chain/01-vcs/green-value-chains/.venv/lib/python3.11/site-packages/posted/utils/utils.py:13: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  ret = pd.merge(
/home/philippv/Documents/4-projects/02-green-value-chain/01-vcs/green-value-chains/.venv/lib/python3.11/site-packages/posted/utils/utils.py:13: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  ret = pd.merge(
/home/philippv/Documents/4-projects/02-green-value-chain/01-vcs/green-value-chains/.venv/lib/python3.11/site-packages/posted/utils/utils.py:13: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  ret = pd.merge(
/home/philippv/Documents/4-projects/02-green-value-chain/01-vcs/green-value-chains/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value enc

### Plot outputs

In [34]:
import plotly.express as px


typeMapping = {
    r'^cap$': 'capital',
    r'^fop$': 'fonmco',
    r'^transp_cost:.*$': 'transport',
    r'^dem_cost:elec$': 'elec',
    r'^dem_cost:(coal|ng)$': 'energy',
    r'^dem_cost:\b(?!(elec|coal|ng))\b.*$': 'rawmat',
}

cost_types = {
    'capital': {
        'label': 'Annualised<br>CAPEX',
        'colour': '#f36f21',
    },
    'fonmco': {
        'label': 'Operation and<br>maintenance',
        'colour': '#00813e',
    },
    'rawmat': {
        'label': 'Raw materials',
        'colour': '#cccccc',
    },
    'energy': {
        'label': 'Non-elec. energy',
        'colour': '#aaaaaa',
    },
    'elec': {
        'label': 'Electricity',
        'colour': '#e6af2e',
    },
    'transport': {
        'label': 'Transport',
        'colour': '#7c1f8a',
    }
}

for comm, lcox in outputs['lcox'].items():
    # convert lcox table to plot data
    plotData = lcox \
        .data['LCOX'] \
        .pint.dequantify().droplevel('unit', axis=1) \
        .stack(['process', 'type']).to_frame('value') \
        .reset_index() \
        .query(f"epdcase=='medium'") \
        .drop(columns=['epdcase'])

    # map types (so that colors are correct)
    plotData['ptype'] = plotData['type'] \
        .replace(regex=typeMapping) \
        .map({
            ptype: display['label']
            for ptype, display in cost_types.items()
        })

    # plot figure
    fig = px.bar(plotData, x='impsubcase', y='value', color='ptype')

    # set colours
    for item in fig.data:
        item['marker']['color'] = next(t['colour'] for t in cost_types.values() if t['label'] == item['legendgroup'])

    # order cases on x-axis
    fig.update_layout(xaxis=dict(categoryorder='array', categoryarray=['Base Case', 'Case 1A', 'Case 1B', 'Case 2', 'Case 3']))

    display(Markdown(f"**{comm}**"))
    display(fig)

**Steel**

**Urea**

**Ethylene**